# Deploy an MLflow model with SageMaker

## Install MLflow

In [ ]:
!pip install -q mlflow==2.15.1

In [ ]:
!pip install scikit-learn==1.1.3

## Setup environment

In [ ]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = 'http://MLflow-MLFLO-wkPtc81lZFSh-d7e92c410416f265.elb.us-east-1.amazonaws.com' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

### Instructions to Build Your Own MLflow Docker Image to Serve a Model

You can do this on your local Docker daemon.

1. Make sure you have configured AWS credentials on your PC:

    ```sh
    $ aws configure
    AWS Access Key ID [****]: Your-Access-Key-ID
    AWS Secret Access Key [****]: Your-Secret-Access-Key
    Default region name [None]: us-east-1
    Default output format [None]: json
    ```

2. Install Docker Desktop https://www.docker.com/products/docker-desktop/ (ensure your Docker Desktop is up and running before proceeding with the following steps).

3. Install MLflow:

    ```sh
    $ pip install mlflow
    ```

4. Install Boto3:

    ```sh
    $ pip install boto3
    ```

5. Run the following command to build and push the MLflow container. This step will take 20 to 30 minutes to complete and will automatically push the container to Amazon ECR:

    ```sh
    $ mlflow sagemaker build-and-push-container
    ```

Once you have your own Docker image, you can start using it.


In [ ]:
#!aws ecr get-login --no-include-email --registry-ids 146171926552 --region us-east-1

In [ ]:
##MLFlow Docker Setup

In [ ]:
!docker images

In [ ]:
!docker pull 03sarath/mlflow-pyfunc:2.12.2 #Pull Docker images from my registry

In [ ]:
!docker images

In [ ]:
#!docker tag 03sarath/mlflow-pyfunc:2.12.2 <You'r ECR repo ID>
!docker tag 03sarath/mlflow-pyfunc:2.12.2 257949588515.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

In [ ]:
!docker images

In [ ]:

#!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <URI OF YOUR REPO>
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 257949588515.dkr.ecr.us-east-1.amazonaws.com/mlflow

In [ ]:
#!docker push <You'r ECR repo ID>
!docker push 257949588515.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

In [ ]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '257949588515.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2'

## Deploy a SageMaker endpoint with our scikit-learn model

In [ ]:
endpoint_name = 'boston-housing-mlops'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/boston/2'

In [ ]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

# client.create_deployment(
#     name=endpoint_name,
#     model_uri=model_uri,
#     flavor='python_function',
#     config=config
# )


# Use update_deployment to update the existing endpoint with the new model version.
client.update_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

## Predict

In [ ]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[5]]
prediction = client.predict(endpoint_name, df.iloc[[5]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

## Delete endpoint

In [ ]:
client.delete_deployment('boston-housing-mlops', config=config)